In [1]:
import kfp
import os
import requests

from kfp.dsl import Input, Model, component, Dataset, Output, Artifact
from kfp.dsl import InputPath, OutputPath, pipeline, component, PipelineTask
from kfp.components import load_component_from_file

In [2]:
@component(
    base_image="python:3.11",
    packages_to_install=["kubernetes==26.1.0"]
)
def delete_ray_pvc(namespace: str, name: str) -> str:
    from kubernetes import client, config

    # Load cluster credentials
    config.load_incluster_config()

    core_v1 = client.CoreV1Api()

    # Optional delete options—for example, enabling graceful deletion:
    delete_options = client.V1DeleteOptions(
        propagation_policy="Foreground",  # ensures dependent resources are also deleted
        grace_period_seconds=0
    )

    core_v1.delete_namespaced_persistent_volume_claim(
        name=f"pvc-{name}",
        namespace=namespace,
        body=delete_options
    )

    return f"Deletion initiated for PVC 'pvc-{name}' in namespace '{namespace}'."

In [3]:
@component(
    base_image="python:3.11",
    packages_to_install=["kserve==0.13.1", "kubernetes==26.1.0", "tenacity==9.0.0"]
)
def destroy_ray_cluster_yaml(namespace: str, name: str):
    import yaml
    from kubernetes import client, config
    
    config.load_incluster_config()
    api = client.CustomObjectsApi()

    cluster_name = f"{name}-kuberay"

    raycluster = api.delete_namespaced_custom_object(
        group='ray.io',
        version='v1',
        namespace=namespace,
        plural='rayclusters',
        name=cluster_name
    )
    

In [4]:
@pipeline(name='delete-ray-cluster-pipeline')
def delete_ray_cluster_pipeline(namespace: str, name: str):
    delete_pvc_op = delete_ray_pvc(namespace=namespace, name=name)
    delete_op = destroy_ray_cluster_yaml(namespace=namespace, name=name).after(delete_pvc_op)

In [5]:
client = kfp.Client()

kfp.compiler.Compiler().compile(delete_ray_cluster_pipeline, './pipelines-yaml/delete_ray_cluster_pipeline.yaml')

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [6]:
run = client.create_run_from_pipeline_func(delete_ray_cluster_pipeline, arguments={"namespace": "demo-ns", "name": "raycluster"}, enable_caching=False)